Test the trained tflite model on new data. 

In [ ]:
# If everything works, then output should be something like the image below.
# However, the tflite-model-maker used for this project used many 
# old packages, so version conflicts are likely during installation. So
# it's probably best to start from scratch with a current object detection
# library.

import matplotlib.image as mpimg
plt.figure(figsize=(8,8))
img=mpimg.imread('car-white-pred.png') # carotene and white colonies
plt.imshow(img);

In [6]:
#!pip install cryptography --upgrade

In [7]:
#!pip install tflite-model-maker
#!pip install pycocotools

In [67]:
#!pip install tensorflow-addons
#!pip install opencv-python
#!pip install tflite_model_maker

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import math

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [2]:
tf.config.list_physical_devices('GPU')
!python --version
tf.__version__
#!conda env list 

[]

In [5]:
spec = model_spec.get('efficientdet_lite1')
spec.config.num_classes=2              # carotene (orange), white
spec.config.tflite_max_detections=100  # <<< default is 100
spec.config.label_map={1: "car", 2: "white"} 
#print(spec.config)

In [6]:
#!pip uninstall opencv-python-headless==4.5.5.62 --y

In [9]:
#%pwd

In [8]:
import cv2

from PIL import Image

# Load the labels into a list
classes = ['???'] * spec.config.num_classes
label_map = spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
    classes[label_id-1] = label_name


# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Return the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


# A high confidence threshold will prevent most overlaps, but not all.
# boxes_overlap was designed to prevent the same colony from being detected more 
# once.
def boxes_overlap(ymin1, xmin1, ymax1, xmax1, ymin2, xmin2, ymax2, xmax2, IOU_threshold):
    
   # if (xmin1 == xmin2 or ymin1 == ymin2 or xmax1 == xmax2 or ymax1 == ymax2):
   #    # the line cannot have positive overlap
   #     return False
         
    # If one rectangle is on left side of other
    if(xmin1 >= xmax2 or xmax1 <= xmin2):
        return False
 
    # If one rectangle is above other
    if(ymax1 <= ymin2 or ymax2 <= ymin1):
        return False
    
    XA1=xmin1;  XA2=xmax1;   YA1=ymin1;   YA2=ymax2;
    XB1=xmin2;  XB2=xmax2;   YB1=ymin2;   YB2=ymax2
    
    SA = (XA2 - XA1) * (YA2 - YA1)
    SB = (XB2 - XB1) * (YB2 - YB1)
    
    SI = max(0, min(XA2, XB2) - max(XA1, XB1)) * max(0, min(YA2, YB2) - max(YA1, YB1))

    pct_overlap = SI/(SA+SB)   # intersection/union

    if pct_overlap <= IOU_threshold:
        return False
        
    #print(f"The two bounding boxes overlap is {pct_overlap}")
 
    return True


# Puts list of results in ascending order by score
# Returns true if two rectangles overlap
def filter_results(results, bb_threshold):

    results_filtered = []  #{'bounding_box':[], 'score':0, 'class_id': 0 }
    
    mylist =[]
    for ii in range(len(results)):
        mylist.append(results[ii]['score'])

    mylist=np.array(mylist)
    idx = np.argsort(mylist*-1)  # want desceding order, == reverse

    results_sorted = []
    for ii in idx:
        results_sorted.append(results[ii])
        
    results = results_sorted   
    
    #verify order    
#     for ii in range(len(results)):
#         print(results[ii]['score'])
    
    for ii in range( len(results) ):
        ymin1, xmin1, ymax1, xmax1 = results[ii]['bounding_box']
        overlap=False
        
        for jj in range(len(results) ):
            if ii==jj:
                continue
        
            ymin2, xmin2, ymax2, xmax2 = results[jj]['bounding_box']
            
            if boxes_overlap(ymin1, xmin1, ymax1, xmax1, ymin2, xmin2, ymax2, xmax2, bb_threshold)\
                and results[ii]['score'] <= results[jj]['score']:
                    overlap=True
                    break

            
        if overlap == False:    
            results_filtered.append(results[ii])  # for cases of overlap, this box had highest score
                  
    return results_filtered



def detect_objects(interpreter, image, confidence_threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  # Feed the input image to the model
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  ###################################################################################################
  # Get all outputs from the model, NOTE the indices might be different for a different model.tflite 
  # that's odd, but that was the fix
  ####################################################################################################
  boxes = get_output_tensor(interpreter, 1)
  classes = get_output_tensor(interpreter, 3)
  scores = get_output_tensor(interpreter, 0)
  count = int(get_output_tensor(interpreter, 2))


  print(">>", scores.size)


  results = []
  for i in range(scores.size):
    if scores[i] >= confidence_threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
        
  print('first box: ', boxes[0])
  print('result count: ', len(results))
    
  return results



def run_odt_and_draw_results(image_path, interpreter, filter_boolean, confidence_threshold, IOU_threshold):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(image_path,
      (input_height, input_width)
    )
    
  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, confidence_threshold)

#   if len(results) == 0:
#     print("no results")
#     return (preprocessed_image, original_uint8)
    
  if filter_boolean:
    print("filtering=True")
    results = filter_results(results, IOU_threshold)
    
  print("length of filtered results: ", len(results))

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)

  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])
    
    print("ymin,xmin,ymax,xmax:", ymin,xmin,ymax,xmax )
    
    if xmin > xmax:
        print("xmin > xmax")
            
    if ymin > ymax:
        print("ymin > ymax")
    
    if abs(xmax - xmin) < 1 or abs(ymax-ymin) < 1: # too small to be a valid colony
        continue

    ## for debugging
    #print(obj['score'] * 100, "==>", xmin,xmax,ymin,ymax,"\n")

    # Find the class index of the current object
    class_id = int(obj['class_id'])
    #print(class_id) # 0, 1, ...

    # Draw the bounding box and label on the image
    color = 1 # [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 1)
    # Make adjustments to make the label visible for all objects
    if (ymin - 7) > 10: 
        y = ymin - 7     
    else: 
        y = ymin + 40
        
        
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)

    #label = "{}: {:.0f}%".format('col', obj['score'] * 100)
    
    cv2.putText(original_image_np, label, (xmin, y), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.85, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return (preprocessed_image, original_uint8)

In [9]:
!ls data-set-maker

01_rotate_images.py  04_norm_coords_and_format.py  images-26-orange-white
02_label_to_csv.py   images-25-26-orange-white
03_rotate_boxes.py   images-25-orange-white


In [10]:
!ls data-set-maker/images-25-26-orange-white/test 

180.jpg			     img_2021-08-25_16-48-12.jpg
270.jpg			     img_2021-08-26_15-48-28.jpg
90.jpg			     img_2021-08-26_15-50-13.jpg
img_2021-08-25_16-47-08.jpg  img_2021-08-26_15-52-08.jpg
img_2021-08-25_16-47-42.jpg  img_2021-08-26_15-52-42.jpg


In [11]:

# train_data, validation_data, test_data = object_detector.DataLoader.from_csv('train_labels-normed.csv')


In [12]:
# iterate to get each test image

In [23]:
#@title Run object detection and show the detection results

# If trained using only a small number of epochs (~5 or 10), classification will be bad, 
# probably need between 50 and 100 epochs (100 takes about 1 hour?).

# If more training, can set the detection (confidence) threshold higher than 0.15.
# Basically, there is a trade-off between true positives and false positives, etc.
#

IOU_threshold=0.10        # intersection/union
confidence_threshold=0.10 # <<<<<<<<<<<================== confidence!
filter_boolean=True       # <<<<<<<<<<<================== excludes ovelapping boxes

test_folder="data-set-maker/images-25-26-orange-white/test/"

test_files= os.listdir(test_folder)

test_path= test_folder + test_files[2]  # <<<=================== choose test file

print(test_path, "\n")

model_path = 'model_50.tflite'      # <<<==== using trained model, but check this

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
(preprocessed_image, detection_result_image) = run_odt_and_draw_results(
    test_path,
    interpreter,
    filter_boolean,
    confidence_threshold,
    IOU_threshold
)


data-set-maker/images-25-26-orange-white/test/90.jpg 

>> 100
first box:  [0.50275904 0.94787407 0.5587607  0.998448  ]
result count:  24
filtering=True
length of filtered results:  7
ymin,xmin,ymax,xmax: 804 1516 894 1597
ymin,xmin,ymax,xmax: 906 1045 985 1124
ymin,xmin,ymax,xmax: 576 1402 662 1492
ymin,xmin,ymax,xmax: 274 1174 355 1258
ymin,xmin,ymax,xmax: 573 1208 656 1293
ymin,xmin,ymax,xmax: 512 1014 591 1093
ymin,xmin,ymax,xmax: 443 1241 522 1322


In [7]:
#img=preprocessed_image[0,:,:,:]/255
#temp=plt.imshow(img)
#print(type(img))
#tf.keras.preprocessing.image.save_img('temp.png',img)

In [6]:
# Show the detection result

#result_img=Image.fromarray(detection_result_image)
#plt.figure(figsize=(10,10))
#temp=plt.imshow(result_img)

In [115]:
#result_img.save("car-white-2.png")